# b01. Pull Scripts from Gitlab
## Pull the Required Scripts Which Will be Used for Model Retraining and Scoring 

---

## Reference
1. [Python gitlab Package - Overview](https://github.com/python-gitlab/python-gitlab)
2. [Python gitlab Package - Getting Started](https://python-gitlab.readthedocs.io/en/stable/api-usage.html?highlight=get%20started)
3. [Gitlab Docs - API resources](https://docs.gitlab.com/ee/api/api_resources.html)
4. [Python sasctl Package - Overview](https://github.com/sassoftware/python-sasctl)
5. [Python sasctl Package - User Guide](https://sassoftware.github.io/python-sasctl/index.html)
6. [SAS Model Manager - REST APIs](https://developer.sas.com/apis/rest/DecisionManagement/)
7. [SAS Model Manager - REST APIs Examples](https://github.com/sassoftware/devsascom-rest-api-samples/tree/master/DecisionManagement)

### 1. 取得特定模型的gitlab access token

In [55]:
#SAS API
import sasctl
from sasctl import Session, register_model
from sasctl.services import model_repository

#Gitlab API
import gitlab

In [69]:
model_name = 'Sklearn Decision Tree'
model_property = 'gitlab-access-token'

with Session('10.249.6.39', 'modelImplementor', 'demopw', verify_ssl = False):    
    model_list = model_repository.list_models()
    
    model_id = next((model.get('id') for model in model_list if model.get('name') == model_name), None)
    
    model = model_repository.get_model(item = model_id)
    
    user_defined_properties = model.get('properties')

    gitlab_access_token = next((item.get('value') for item in user_defined_properties if item.get('name') == model_property), None)


print(f'{model}\'s model ID: {model_id}\n')
print(f'{model_property}: {gitlab_access_token}\n')

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.249.6.39'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


Sklearn Decision Tree's model ID: 4c9e48c6-83fe-496c-9acd-692576f48b29

gitlab-access-token: ynJH8ppLyiB_2L5sT2NY



### 2. Pull gitlab的retrain.py、score.py

In [56]:
project_name = 'demo project 1'
gl = gitlab.Gitlab(url = 'https://gitlab.com/', private_token = gitlab_access_token)
project_list = gl.projects.list(owned = True)
project_id = next((project.id for project in project_list if project.name == project_name), None)
project = gl.projects.get(project_id)

file_list = project.repository_tree()
retrain_py = next((file for file in file_list if file.get('name') == 'retrain.py'), None)
retrain_py_id = retrain_py.get('id')
retrain_py_blob = project.repository_raw_blob(sha = retrain_py_id)

score_py = next((file for file in file_list if file.get('name') == 'score.py'), None)
score_py_id = score_py.get('id')
score_py_blob = project.repository_raw_blob(sha = score_py_id)

[{'id': '3389bb0aac55af218f7a0a0d3593a4e7f3e942f1', 'name': 'README.md', 'type': 'blob', 'path': 'README.md', 'mode': '100644'}, {'id': 'cdbc8c9cfe5da62c79348111a21c43fec47b37cb', 'name': 'retrain.py', 'type': 'blob', 'path': 'retrain.py', 'mode': '100644'}, {'id': 'eb9f154d68be8d6b844debe30ae0b7385265a0d8', 'name': 'score.py', 'type': 'blob', 'path': 'score.py', 'mode': '100644'}]

### 3. retrain.py、score.py傳至model repository

In [78]:
with Session('10.249.6.39', 'modelImplementor', 'demopw', verify_ssl = False):    
    model_list = model_repository.list_models()
    model_id = next((model.get('id') for model in model_list if model.get('name') == model_name), None)
    model = model_repository.get_model(item = model_id)

    model_repository.add_model_content(
        model = model,
        file = retrain_py_blob,
        name = 'retrain.py',
        role = 'Python Script'
    )

    model_repository.add_model_content(
        model = model,
        file = score_py_blob,
        name = 'score.py',
        role = 'Python Script'
    )

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.249.6.39'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
